# Cleaning Many Datasets

In [1]:
import dask, dask.dataframe as dd
import matplotlib.pyplot as plt
import pandas as pd
import re, csv, os
import numpy as np
from dask import delayed, persist
from glob import glob

pd.set_option('display.max_columns', None)
csv.field_size_limit(10000000)

%matplotlib inline

## What to do before running these cells

Add the path to your files to the variable **path** below.

For **partitions_out** below think about how many GB you will be cleaning and how many files you will like to have at the end of the cleaning process. A good rule of thumb is to split large files into manageable chunks of 300 to 600 MB for analysis. If you would like to follow this approach, figure out how much data you will be cleaning in MB terms (1 GB = 1000 MB) and divide it by the size in MB terms that you would like your final files to have. For example, 3GB (or 3,000MB) divided by 300MB would amount to 10 partitions.

For **partitions_in**, do something somewhat similar than with **partitions_out** but to a much larger scale. If you are cleaning 100 GB of data make about 1000 partitions so that dask can clean faster with very manageable chunks.

In [2]:
path = '/Volumes/LaCie SSD/bgdata/data_19/some_data/new_test'
partitions_in = 15
partitions_out = 10

The following are the variables which I've determined the most useful. Feel free to add or subtract from them before running the cells below. No need to update the `dtypes` dictionary below as it contains all the variables in the BG dataset.

In [3]:
best_list = ['JobID', 'CleanJobTitle', 'CanonCity', 'CanonState', 'JobDate', 'JobText', 'Source', 'CanonEmployer',
             'Latitude', 'Longitude', 'CanonIntermediary', 'CanonJobTitle', 'CanonCounty', 'DivisionCode', 'MSA', 'LMA',
             'InternshipFlag', 'ConsolidatedONET', 'CanonSkillClusters', 'CanonSkills', 'IsDuplicate', 'CanonMinimumDegree', 
             'CanonRequiredDegrees', 'CIPCode', 'MinExperience', 'ConsolidatedInferredNAICS', 'BGTOcc', 'MaxAnnualSalary',
             'MaxHourlySalary', 'MinAnnualSalary', 'MinHourlySalary', 'YearsOfExperience', 'CanonJobHours', 'CanonJobType',
             'CanonPostalCode', 'CanonYearsOfExperienceCanonLevel', 'CanonYearsOfExperienceLevel', 'ConsolidatedTitle', 
             'Language', 'BGTSubOcc', 'ConsolidatedDegreeLevels', 'MaxDegreeLevel', 'MinDegreeLevel']

Because the data is very messy and dask can't infer correctly all of the variables' data types without taking away the gain of parallelizing the computations, we will import every var with the data type as a string.

In [4]:
dtypes={'JobID': np.str, 'CleanJobTitle': np.str, 'JobDomain': np.str, 
        'CanonCity': np.str, 'CanonCountry': np.str, 'CanonState': np.str, 
        'JobText': np.str, 'JobURL': np.str, 'PostingHTML': np.str, 
        'Source': np.str, 'JobReferenceID': np.str, 'Email': np.str, 
        'CanonEmployer': np.str, 'Latitude': np.str, 'Longitude': np.str, 
        'CanonIntermediary': np.str, 'Telephone': np.str, 'CanonJobTitle': np.str, 
        'CanonCounty': np.str, 'DivisionCode': np.str, 'MSA': np.str, 'LMA': np.str,
        'InternshipFlag': np.str, 'ConsolidatedONET': np.str, 'CanonCertification': np.str, 
        'CanonSkillClusters': np.str, 'CanonSkills': np.str, 'IsDuplicate': np.str, 
        'IsDuplicateOf': np.str, 'CanonMaximumDegree': np.str, 'CanonMinimumDegree': np.str, 
        'CanonOtherDegrees': np.str, 'CanonPreferredDegrees': np.str,
        'CanonRequiredDegrees': np.str, 'CIPCode': np.str, 'StandardMajor': np.str, 
        'MaxExperience': np.str, 'MinExperience': np.str, 'ConsolidatedInferredNAICS': np.str, 
        'BGTOcc': np.str, 'MaxAnnualSalary': np.str, 'MaxHourlySalary': np.str, 
        'MinAnnualSalary': np.str, 'MinHourlySalary': np.str, 'YearsOfExperience': np.str, 
        'CanonJobHours': np.str, 'CanonJobType': np.str, 'CanonPostalCode': np.str, 
        'CanonYearsOfExperienceCanonLevel': np.str, 'CanonYearsOfExperienceLevel': np.str, 
        'ConsolidatedTitle': np.str, 'Language': np.str, 'BGTSubOcc': np.str, 'JobDate': np.str,
        'ConsolidatedDegreeLevels': np.str, 'MaxDegreeLevel': np.str, 'MinDegreeLevel': np.str,
                       }

Notice the wildcard in the `os.path.join()` call of your dask dataframe `read_csv` function. That tells Dask to grab all of the files that end with `'.csv'` inside your folder. You can make it more specific by adding more characters before and after the star. For example, `'data_0*.csv'` will grab all CSV files in your folder that start with `data_0` and end with `.csv`.

Also notice the we pass in the list of variables and the the dictionary of data types. We also tell dask to assume that there will be missing data with the parameter `assume_missing`. Error bad lines will print the bad lines that dask skips for us.

Make sure to add a few letters from the start of your files.

Now run everything and wait. :)

In [5]:
ddf = dd.read_csv(os.path.join(path, 'da*.csv'), 
                 engine='python', 
                 dtype=dtypes,
                 assume_missing=True,
                 error_bad_lines=False,
                 blocksize=None,
                 usecols=best_list,
                )
ddf

,JobID,CleanJobTitle,CanonCity,CanonState,JobDate,JobText,Source,CanonEmployer,Latitude,Longitude,CanonIntermediary,CanonJobTitle,CanonCounty,DivisionCode,MSA,LMA,InternshipFlag,ConsolidatedONET,CanonSkillClusters,CanonSkills,IsDuplicate,CanonMinimumDegree,CanonRequiredDegrees,CIPCode,MinExperience,ConsolidatedInferredNAICS,BGTOcc,MaxAnnualSalary,MaxHourlySalary,MinAnnualSalary,MinHourlySalary,YearsOfExperience,CanonJobHours,CanonJobType,CanonPostalCode,CanonYearsOfExperienceCanonLevel,CanonYearsOfExperienceLevel,ConsolidatedTitle,Language,BGTSubOcc,ConsolidatedDegreeLevels,MaxDegreeLevel,MinDegreeLevel
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [6]:
# here is where we repartition our data
ddf00 = ddf.repartition(npartitions=partitions_in)

In [7]:
%%time

ddf00.head()

CPU times: user 1min 29s, sys: 2min 36s, total: 4min 5s
Wall time: 7min 47s


,JobID,CleanJobTitle,CanonCity,CanonState,JobDate,JobText,Source,CanonEmployer,Latitude,Longitude,CanonIntermediary,CanonJobTitle,CanonCounty,DivisionCode,MSA,LMA,InternshipFlag,ConsolidatedONET,CanonSkillClusters,CanonSkills,IsDuplicate,CanonMinimumDegree,CanonRequiredDegrees,CIPCode,MinExperience,ConsolidatedInferredNAICS,BGTOcc,MaxAnnualSalary,MaxHourlySalary,MinAnnualSalary,MinHourlySalary,YearsOfExperience,CanonJobHours,CanonJobType,CanonPostalCode,CanonYearsOfExperienceCanonLevel,CanonYearsOfExperienceLevel,ConsolidatedTitle,Language,BGTSubOcc,ConsolidatedDegreeLevels,MaxDegreeLevel,MinDegreeLevel
0,38674096361,Host,Smyrna,TN,2019-12-03,"HOST\n\nDarden Restaurants, Inc. Smyrna Full-T...",Job Board,Darden Restaurants Incorporated,35.9651,-86.5147,NaN,Host/Hostess,Rutherford,NaN,34980: Metropolitan Statistical Area,MT473498,0,35903100,Customer and Client Support: Cash Register Ope...,{'Cash Register Operation': 'Customer and Clie...,FALSE,NaN,NaN,NaN,NaN,722,35-9031.00,38000.00,18.27,38000.00,18.27,NaN,fulltime,permanent,37167,NaN,NaN,Host/Hostess,en,Host / Hostess,NaN,NaN,NaN
1,38674096395,Delivery Driver,Natick,MA,2019-12-03,Delivery Driver - Natick\n\nPanera Bread FDF N...,Job Board,Panera Bread,42.2858,-71.3551,NaN,Delivery Driver,Middlesex,73104,71650: Metropolitan NECTA|715: Combined NECTA,DV257310|MT257165,0,53303100,Specialized Skills,{'Product Delivery': 'Specialized Skills'},FALSE,NaN,NaN,NaN,NaN,722513,53-3031.00,50000.00,24.04,50000.00,24.04,NaN,fulltime,permanent,01760,NaN,NaN,Delivery Driver,en,Restaurant Delivery Driver,NaN,NaN,NaN
2,38674096626,Host,Bear,DE,2019-12-03,"HOST\n\nDarden Restaurants, Inc. Bear Full-Tim...",Job Board,Darden Restaurants Incorporated,39.6103,-75.6896,NaN,Host/Hostess,New Castle,48864,37980: Metropolitan Statistical Area|428: Comb...,DV104886|MT423798,0,35903100,Customer and Client Support: Cash Register Ope...,{'Cash Register Operation': 'Customer and Clie...,FALSE,NaN,NaN,NaN,NaN,722,35-9031.00,41357.00,19.88,41357.00,19.88,NaN,fulltime,permanent,19701,NaN,NaN,Host/Hostess,en,Host / Hostess,NaN,NaN,NaN
3,38674096471,Server,Clinton Township,MI,2019-12-03,"SERVER\n\nDarden Restaurants, Inc. Clinton Tow...",Job Board,Darden Restaurants Incorporated,42.5772,-82.9098,NaN,NaN,Macomb,47664,19820: Metropolitan Statistical Area|220: Comb...,DV264764|MT261982,0,35303100,NaN,{},FALSE,NaN,NaN,NaN,NaN,722,35-3031.00,46687.00,22.45,46687.00,22.45,NaN,fulltime,permanent,48035,NaN,NaN,Server,en,Waiter / Waitress,NaN,NaN,NaN
4,38674096550,Nursing - Non-Certified Nursing Assistant,Wyncote,PA,2019-12-03,Nursing - Non-Certified Nursing Assistant\n\nG...,Job Board,Genesis Healthcare Corporation,40.0872,-75.1503,NaN,Certified Nursing Assistant,Montgomery,33874,37980: Metropolitan Statistical Area|428: Comb...,DV423796|MT423798,0,31101400,Health Care: Basic Living Activities Support;S...,{'Activities of Daily Living (ADLS)': 'Health ...,FALSE,Higher Secondary Certificate,High School Equivalency|Higher Secondary Certi...,NaN,NaN,62311,31-1014.00,86248.00,41.47,86248.00,41.47,NaN,NaN,NaN,19095,NaN,NaN,Certified Nursing Assistant,en,Nursing Assistant,12,NaN,12


In [8]:
%%time

ddf00.tail()

CPU times: user 1min 46s, sys: 5min 43s, total: 7min 29s
Wall time: 17min 26s


,JobID,CleanJobTitle,CanonCity,CanonState,JobDate,JobText,Source,CanonEmployer,Latitude,Longitude,CanonIntermediary,CanonJobTitle,CanonCounty,DivisionCode,MSA,LMA,InternshipFlag,ConsolidatedONET,CanonSkillClusters,CanonSkills,IsDuplicate,CanonMinimumDegree,CanonRequiredDegrees,CIPCode,MinExperience,ConsolidatedInferredNAICS,BGTOcc,MaxAnnualSalary,MaxHourlySalary,MinAnnualSalary,MinHourlySalary,YearsOfExperience,CanonJobHours,CanonJobType,CanonPostalCode,CanonYearsOfExperienceCanonLevel,CanonYearsOfExperienceLevel,ConsolidatedTitle,Language,BGTSubOcc,ConsolidatedDegreeLevels,MaxDegreeLevel,MinDegreeLevel
718274,38677985774,Crew Member,North Attleborough,MA,2019-12-09,Crew Member\n\nDunkin' Donuts\n\n-\n\nNorth At...,Job Board,Dunkin' Donuts,41.9748,-71.3264,NaN,NaN,Bristol,NaN,77200: Metropolitan NECTA|715: Combined NECTA,MT447720,0,35302100,Personal Care and Services: Food and Beverage ...,{'Restaurant Experience': 'Personal Care and S...,FALSE,NaN,NaN,NaN,NaN,722515,35-3021.00,NaN,NaN,NaN,NaN,NaN,fulltime,permanent,02760,NaN,NaN,Crew Member,en,Food Service Team Member,NaN,NaN,NaN
718275,38677985779,Office Coordinator,Peoria,AZ,2019-12-09,Office Coordinator\n\nEPIC Home Realty\n\nPeor...,Job Board,NaN,33.5696,-112.248,NaN,Office Coordinator,Maricopa,NaN,38060: Metropolitan Statistical Area,MT043806,0,43601400,Design: Graphic and Visual Design Software;Spe...,{'Adobe Photoshop': 'Design: Graphic and Visua...,FALSE,NaN,NaN,NaN,NaN,53,43-6014.00,NaN,NaN,NaN,NaN,NaN,fulltime,permanent,85345,NaN,NaN,Office Coordinator,en,Administrative Clerk / Coordinator,NaN,NaN,NaN
718276,38677985797,Supervisor,Los Angeles,CA,2019-12-09,Supervisor\n\nLA Times | Areas USA\n\n-\n\nLos...,Job Board,La Times Areas Usa,33.9746,-118.247,NaN,NaN,Los Angeles,31084,31080: Metropolitan Statistical Area|348: Comb...,DV063108|MT063110,0,35101200,Administration: Office Machines;Specialized Sk...,{'Adding Machine': 'Administration: Office Mac...,FALSE,Higher Secondary Certificate,NaN,NaN,NaN,7225,35-1012.00,NaN,NaN,NaN,NaN,One to two years,fulltime,permanent,90001,NaN,NaN,Supervisor,en,Restaurant / Food Service Supervisor,14|12,14,12
718277,38677985812,Npdes Coordinator,Tampa,FL,2019-12-09,NPDES Coordinator\n\nLevel Experienced Job Loc...,Company,Enviromental Science Associates,27.9827,-82.3402,NaN,NaN,Hillsborough,NaN,45300: Metropolitan Statistical Area,MT124530,0,11911100,Finance: Financial Reporting;Specialized Skill...,{'Annual Reports': 'Finance: Financial Reporti...,FALSE,Bachelor's,Bachelor's,301801,5,NaN,NaN,NaN,NaN,NaN,NaN,minimum of 5 years,fulltime,permanent,33601,1-6,mid,Coordinator,en,NaN,16,NaN,16
718278,38677986013,Dietitian/Nutrition Care Manager,Fayetteville,NC,2019-12-09,Dietitian / Nutrition Care Manager - Pinehurst...,Job Board,Compass Group Usa Incorporated,35.0016,-78.8309,NaN,Dietitian,Cumberland,NaN,22180: Metropolitan Statistical Area,MT372218,0,11911100,Health Care: Nutrition and Diet;Specialized Sk...,{'Dietetics': 'Health Care: Nutrition and Diet...,FALSE,Associate's,Associate's,NaN,1,6233,29-1141.91,NaN,NaN,NaN,NaN,1) year,fulltime,permanent,28301,0-1,low,Dietitian,en,Clinical Case Manager,14,NaN,14


In [9]:
ddf00.npartitions

15

In [10]:
# There are missing company names that map to a recruiting agency and because of this
# we will identify those observations and fill in the missing valyes in the CanonEmployer
# var with "Recruitment Agency"
EmployerCondition = ((ddf00['CanonEmployer'].isnull()) & (ddf00['CanonIntermediary'].notnull()))
EmployerClean = ddf00['CanonEmployer'].where(~EmployerCondition, 'Recruitment Agency')

# we will then drop the original variable and add the new one to the dataset using the following methods
ddf_clean0 = ddf00.drop('CanonEmployer', axis=1)
ddf_clean01 = ddf_clean0.assign(EmployerClean=EmployerClean)

### Note: The following cell will take a while!

In [9]:
%%time

# We have a lot of missing values in this dataset so let's start by calculating those
# as a percentage of all of the samples in our datasets
missing_count = ((ddf_clean01.isna().sum() / ddf_clean01.index.size) * 100)
missing_count_pct = missing_count.compute()
missing_count_pct

CPU times: user 3min 1s, sys: 8min 36s, total: 11min 37s
Wall time: 22min 48s


JobID                                0.000000
CleanJobTitle                        0.005836
CanonCity                            1.201399
CanonState                           0.052183
JobDate                              0.000000
JobText                              0.000085
Source                               0.585603
Latitude                             1.150653
Longitude                            1.150653
CanonIntermediary                   89.087614
CanonJobTitle                       37.319558
CanonCounty                          1.204951
DivisionCode                        68.001272
MSA                                  3.173536
LMA                                  1.431275
InternshipFlag                       0.000000
ConsolidatedONET                     4.074099
CanonSkillClusters                   4.407919
CanonSkills                          0.000000
IsDuplicate                          0.000000
CanonMinimumDegree                  46.179629
CanonRequiredDegrees              

In [10]:
# we will now drop the columns with 60% or more missing values
cols_to_drop = list(missing_count_pct[missing_count_pct >= 60].index)
cols_to_drop

['CanonIntermediary',
 'DivisionCode',
 'CIPCode',
 'MaxAnnualSalary',
 'MaxHourlySalary',
 'MinAnnualSalary',
 'MinHourlySalary',
 'MaxDegreeLevel']

In [11]:
# Since the rows above have more than 60, 70 and 80% of missing values,
# we will be getting rid of them with the drop command
ddf_clean1 = ddf_clean01.drop(cols_to_drop, axis=1)

# since english must be the most common language for the majority of positions in 
# the USA, we will fill in missing values in that colunm with the en value in the Language var
ddf_clean2 = ddf_clean1.fillna({'Language': 'en'})

In [12]:
# here we will get rid of the rows in columns with missing values 
# between 1 and 10%
rows_to_drop = list(missing_count_pct[(missing_count_pct < 10) & (missing_count_pct > 0)].index)
rows_to_drop

['CleanJobTitle',
 'CanonCity',
 'CanonState',
 'JobText',
 'Source',
 'Latitude',
 'Longitude',
 'CanonCounty',
 'MSA',
 'LMA',
 'ConsolidatedONET',
 'CanonSkillClusters',
 'BGTOcc',
 'CanonPostalCode',
 'ConsolidatedTitle',
 'BGTSubOcc',
 'EmployerClean']

In [13]:
# here is the code to drop them
ddf_clean3 = ddf_clean2.dropna(subset=rows_to_drop)

In [14]:
# we will assign the word "Unknown" the remaining columns with missing values
# The nice thing about python and many other languages is that we can read the data
# and tell it to reassign np.nan to observations containing the word "Unknown"
remaining_cols_to_clean = list(missing_count_pct[(missing_count_pct >= 10) & (missing_count_pct < 60)].index)
unknown_default_dict = dict(map(lambda columnName: (columnName, 'Unknown'), remaining_cols_to_clean))
unknown_default_dict

{'CanonJobTitle': 'Unknown',
 'CanonMinimumDegree': 'Unknown',
 'CanonRequiredDegrees': 'Unknown',
 'MinExperience': 'Unknown',
 'ConsolidatedInferredNAICS': 'Unknown',
 'YearsOfExperience': 'Unknown',
 'CanonJobHours': 'Unknown',
 'CanonJobType': 'Unknown',
 'CanonYearsOfExperienceCanonLevel': 'Unknown',
 'CanonYearsOfExperienceLevel': 'Unknown',
 'ConsolidatedDegreeLevels': 'Unknown',
 'MinDegreeLevel': 'Unknown'}

In [15]:
# here we fill in those missing values
ddf_clean4 = ddf_clean3.fillna(unknown_default_dict)

In [16]:
# if you would like to make sure you don't have any other missing values,
# uncomment and run the cell below

# print(ddf_clean4.isnull().sum().compute())

In [17]:
# The JobText var is not formatted correctly so we will first clean it
# and create a new variable called clean_text
clean_text = ddf_clean4.loc[:, 'JobText'].str.strip()
# .apply(lambda x: ' '.join(list(filter(None, x.split()))), meta=np.str)

# we will then drop the old JobText var
ddf_clean5 = ddf_clean4.drop('JobText', axis=1)

# Here we reassign the cleaned var back into the dataset
ddf_clean6 = ddf_clean5.assign(clean_text=clean_text)

# we will now filter out job descriptions that are not written in english
english_condition = ddf_clean6['Language'].isin(['en'])
ddf_clean7 = ddf_clean6[english_condition]

# We will then convert the JobDate var into a date variable
dates = dd.to_datetime(ddf_clean7['JobDate'])
# drop the old one
ddf_clean8 = ddf_clean7.drop('JobDate', axis=1)
# and reassign the new one
ddf_clean9 = ddf_clean8.assign(JobDate=dates)

# let's filter out duplicate jobs and then drop that column
no_duplicates = (ddf_clean9['IsDuplicate'] == "FALSE")
ddf_clean10 = ddf_clean9[no_duplicates]
ddf_clean11 = ddf_clean10.drop(['IsDuplicate', 'Language'], axis=1)

### Note: The following cell will take a while!

In [18]:
%%time

# using the same folder in your path, we will create a new one for the cleaned data
# and save our new files there
if not os.path.exists(os.path.join(path, 'clean')):
    os.makedirs(os.path.join(path, 'clean'))
    

# the following lines of code will take the last dataset, repartition it,
# and save it to the desired location. Notice the wildcard "*" below. That is
# the spot Dask will use to number your files starting from 0
(ddf_clean11
 .repartition(npartitions=partitions_out)
 .to_csv(os.path.join(path, 'clean/', 'data_cleaned_*.csv'), index=False)
 )

CPU times: user 6min 41s, sys: 21min 2s, total: 27min 43s
Wall time: 56min 42s


['/Volumes/LaCie SSD/bgdata/data_19/some_data/clean/data_cleaned_00.csv',
 '/Volumes/LaCie SSD/bgdata/data_19/some_data/clean/data_cleaned_01.csv',
 '/Volumes/LaCie SSD/bgdata/data_19/some_data/clean/data_cleaned_02.csv',
 '/Volumes/LaCie SSD/bgdata/data_19/some_data/clean/data_cleaned_03.csv',
 '/Volumes/LaCie SSD/bgdata/data_19/some_data/clean/data_cleaned_04.csv',
 '/Volumes/LaCie SSD/bgdata/data_19/some_data/clean/data_cleaned_05.csv',
 '/Volumes/LaCie SSD/bgdata/data_19/some_data/clean/data_cleaned_06.csv',
 '/Volumes/LaCie SSD/bgdata/data_19/some_data/clean/data_cleaned_07.csv',
 '/Volumes/LaCie SSD/bgdata/data_19/some_data/clean/data_cleaned_08.csv',
 '/Volumes/LaCie SSD/bgdata/data_19/some_data/clean/data_cleaned_09.csv',
 '/Volumes/LaCie SSD/bgdata/data_19/some_data/clean/data_cleaned_10.csv',
 '/Volumes/LaCie SSD/bgdata/data_19/some_data/clean/data_cleaned_11.csv',
 '/Volumes/LaCie SSD/bgdata/data_19/some_data/clean/data_cleaned_12.csv',
 '/Volumes/LaCie SSD/bgdata/data_19/so